In [6]:
import requests, time, pickle
from bs4 import BeautifulSoup
import re

In [14]:
with open("headers.txt", 'r') as f:
    txt = f.read()
    json = txt.split("\n")
    headers = {}
    for i in json:
        headers[i.split(":")[0].strip()] = i.split(":")[1].strip()
    f.close()
headers

{'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
 'Accept-Encoding': 'gzip, deflate, br',
 'Accept-Language': 'zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7',
 'Cache-Control': 'max-age=0',
 'Connection': 'keep-alive',
 'Cookie': 'session-id=460-4720087-2775234; ubid-acbcn=462-3710794-7122163; lc-acbcn=zh_CN; x-wl-uid=12xpjk158hAQqvhEriGwb66HlklWHF5XgjEiOLwkR3DLe8gwyXrzApK5jRB7uBOz2fDd/QXKcLvge5FwH26Go+FvZn1vEh9qriGZPbrCpeQNV6WOa7YvVyKZRhzQyX5NToj7I5oBXQM0=; x-acbcn="H384wQ7ENOfsd6sLaMWPQC42Xkbu4jX?Uj6Pwz@avnhZvY1ZITifJeEuFusLL9A1"; at-main=Atza|IwEBIH4uB3v2fS537KL6j731VU4N_bXEA8bA5nFh2dqRmZWTHEScqBCjmArFsTZJiuz_Z044DRumtEVpu-amYJvkJ4dwYyPisEIai3jlNWpvc6f7dVegx5EtoMZ0Nfc_AQ3mX5Sfu8xIlXF1-UR3otLVcaZHDEXcwc8-wBZLctit_yqPMhXewRxDK_WV7g_5cX6_-lJ0hFUxIik0bKYi6yhMKn9srS3aJZ28Zga7__VX9VITD8o4NZx5S61sqXXx1DSVrrlOd3fZS4UP-khU8Jb7exdpch8HcUPgTZ4c8OyYhslLKP17zaxjg3brFpv_PvlYM4kl9Cduo-iBLd1Sq7ESOhSHy0Q6fBDCJqBb_uiAr3naF909gKlHpA_FKkgpkW-tkf6ugYM3rJAGv-52OyE

In [98]:
urls = ['https://www.amazon.cn/dp/B00HNQ7I18/?coliid=I1T9Z4T81V739X&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B00E37QBDG/?coliid=I2FVDURN4T06TE&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B019C5Y1YO/?coliid=I3DQARI3U2ETYY&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B0015BUZSU/?coliid=I2S7WQOB1CKTKJ&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B01N4K7QQW/?coliid=I315PEDV9N0JFW&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B01IEXW886/?coliid=I2EP7ZWPLIFY3T&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B06XPFBG6G/?coliid=I266KSQT89IJLR&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B00ZR3MOYK/?coliid=I2E8Y9WM57ADNM&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B00H430SFS/?coliid=I12OKMRHHX2TA&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B01N1MV8FT/?coliid=I2R29V5X7MWU2U&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B0711Q56JF/?coliid=I2B1QMRFC4KZBU&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B00HVSWERQ/?coliid=I3AUCJYCYXWH6X&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B078NPR8C7/?coliid=I1Y4Z6FPHYC1DA&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B00KW8A4CO/?coliid=IJIFGTS63PVIQ&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B0083F4WTO/?coliid=I2JT8LY4X2OOIW&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B078MBLZVS/?coliid=I1Q9Z6G63HH666&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B00RG76M3Y/?coliid=IV1EZQVZAIT73&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B01NCJAA12/?coliid=INOKQHJAGIXTT&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B00MPIWA4Y/?coliid=IKZQ6DNOL1I2&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B01742Y31S/?coliid=I20G9PHEG7FSTQ&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B01CF2UAI2/?coliid=I9J0U44575ZPA&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B007E7G9ZC/?coliid=I1MY9YBCPZNLRL&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/1101946598/?coliid=I1MNNZPAN9RHRD&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/0674023382/?coliid=I10PBP0E0E527J&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B001LA1KLQ/?coliid=I1TN697T8WEF0D&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B0011FA5LM/?coliid=I2FEZ63G9N06S2&colid=3OYPXXDLPVYWS&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B00EE5IAHW/?coliid=I19HZJ9CLLJH9A&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B01K4FS4TE/?coliid=I1ED01074WC6L4&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B071K88XPR/?coliid=ID762FIIYQ0J1&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B0716WKH2J/?coliid=I2LW1DZX1F18KZ&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B072K4CHFH/?coliid=I2UMTHR14RQA74&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B076WQK5TW/?coliid=I1LH7RT3JQ6DBV&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B071NZB41D/?coliid=I2WE244E2F20XO&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B01M8MZ5DX/?coliid=I3CWEBVV0UO1EF&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B074C2XWB4/?coliid=I4Z7DTHVDWKCK&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B076DGNXP1/?coliid=IMTROU319MENW&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B076WRJTMN/?coliid=I38L97JVM17RT7&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B015QSNGLU/?coliid=I11APS13RBJG1H&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B00DMS9990/?coliid=ITPBSP1KWV91P&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B073LYQ7WG/?coliid=IPWM3R3A1YEMH&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B075JDNR37/?coliid=I6L6P3WDZPSKT&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B07BHK11WQ/ref=pd_cp_351_1?_encoding=UTF8&pd_rd_i=B07BHK11WQ&pd_rd_r=352a0d76-8689-11e8-bac7-df01cc25df45&pd_rd_w=FB2EB&pd_rd_wg=pv7cH&pf_rd_i=desktop-dp-sims&pf_rd_m=A1AJ19PSB66TGU&pf_rd_p=2731279874357502789&pf_rd_r=C9K67YHRDND2EHS61CZT&pf_rd_s=desktop-dp-sims&pf_rd_t=40701&psc=1&refRID=C9K67YHRDND2EHS61CZT',
 'https://www.amazon.cn/dp/B00ICPHHVA/?coliid=I1CL6IVXINZBDW&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B00C4OM7V0/?coliid=I3BIUA1C5U1SLQ&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it',
 'https://www.amazon.cn/dp/B004JZY9BI/?coliid=I1HV3GTTJOF01V&colid=29F9RD6YIB38K&psc=0&ref_=lv_ov_lig_dp_it']

In [101]:
def get_info():
    book = []
    price = []
    for url in urls:
        time.sleep(0.5)
        r = requests.get(url, headers = headers)
        txt = BeautifulSoup(r.text, 'lxml')
        pattern1 = re.findall(r'roductTitle".*<',str(txt))
        book_name = re.findall(r">(.*)<", str(pattern1))
        print("获取《" + book_name[0] + "》完成")
        pattern2 = re.compile(r'\"a\-size\-base\sa\-color\-price\s.*￥\d{1,4}\.\d{2}', flags=re.DOTALL)
        result2 = (pattern2.findall(str(txt)))
        today_price = float(re.findall(r"\d{1,4}\.\d{2}", result2[0])[0])
        print("价格为：" + str(today_price))
        if book_name in book == True:
            price[price.index(book_name)][1] = today_price
        else:
            book.append(book_name)
            price.append([book_name, price])
    write_to_history(price)

def write_to_history(price):
    with open("yesterday.pk", "rb") as a:
        yesterday = pickle.load(a)
        a.close()
    with open("yesterday.pk", "wb") as f:
        yesterday = dump.load(price, f)
        f.close()
    compare_book(price, yersterday)

def compare_book(today, yersterday):
    compare = today
    for i, j in today, yersterday:
        compare[compare.index(i)][1] = i[1] - j[1]
    for i in compare:
        print(compare[0],"价格差为", compare[1])

get_info()

获取《逻辑学导论 (21世纪大学文科教材,复旦博学·哲学系列)》完成
价格为：9.89
获取《美学导论 (21世纪大学文科教材,复旦博学·哲学系列)》完成
价格为：9.9
获取《重新定义团队：谷歌如何工作（完整图文版）》完成
价格为：12.99
获取《优势谈判》完成
价格为：19.66
获取《微观动机与宏观行为（诺贝尔经济学奖获得者丛书）》完成
价格为：22.8
获取《历史主义贫困论》完成
价格为：23.8
获取《礼物的流动：一个中国村庄中的互惠原则与社会网络》完成
价格为：28.23
获取《西方现代思想丛书10:致命的自负》完成
价格为：29.8
获取《理论思维与工程思维:两种思维方式的僭越与划界(修订本)》完成
价格为：32.3
获取《改变心理学的40项研究》完成
价格为：32.6
获取《天朝的崩溃:鸦片战争再研究(修订版) (三联哈佛燕京学术丛书)》完成
价格为：35.4
获取《工商管理经典译丛·市场营销系列:消费者行为学(第10版)》完成
价格为：53.7
获取《决策心理：齐当别之道》完成
价格为：54.5
获取《超级合作者 (财富汇)》完成
价格为：54.5
获取《网络科学导论》完成
价格为：56.1
获取《原则（完整图文版）》完成
价格为：58.0
获取《复杂网络引论:模型、结构与动力学(英文版)(第二版)》完成
价格为：60.5
获取《开放社会及其敌人(套装共2册)》完成
价格为：99.1


IndexError: list index out of range